In [1]:
dataset_id = "weather_staging"


In [2]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset 'uber-fair:weather_staging' already
exists.


In [3]:
!bq --location=US load --autodetect --skip_leading_rows=1 --source_format=CSV {dataset_id}.weather \
'gs://uber_fair_data/dataset1/weather.csv'

Waiting on bqjob_rd86abbbb638f95d_0000016ed7b9d804_1 ... (1s) Current status: DONE   


In [4]:
%%bigquery 
select avg(price) as price, w.rain
from `uber-fair.rideshare_modeled.Rider`r ,`uber-fair.weather_modeled.weather` w
group by w.rain

,price,rain
0,16.545125,0.0000
1,16.545125,0.0260
2,16.545125,0.1123
3,16.545125,0.0030
4,16.545125,0.1672
...,...,...
433,16.545125,0.1307
434,16.545125,0.1813
435,16.545125,0.0622
436,16.545125,0.1556


In [6]:
%%bigquery 
select avg(price) as price, avg(rain) as rain, r.cab_type
from `uber-fair.rideshare_modeled.Rider`r ,`uber-fair.weather_modeled.weather` w
where w.rain != 0
group by r.cab_type

,price,rain,cab_type
0,17.351396,0.057652,Lyft
1,15.795343,0.057652,Uber


In [8]:
%%bigquery 
select t.temp_level , avg(r.price) price
from (select case 
       when w.temp between 0 and 30 then "0-10"
       when w.temp between 30 and 100 then "30-100"
       else "End Range"
  end as temp_level
  from `uber-fair.weather_modeled.weather` w)t, `uber-fair.rideshare_modeled.Rider` r
  group by t.temp_level
  order by t.temp_level

,temp_level,price
0,0-10,16.545125
1,30-100,16.545125
